In [37]:
import numpy as np
import awkward as ak

import hist
from topcoffea.modules.histEFT import HistEFT
import topcoffea.modules.utils as utils

import mplhep as hep
import matplotlib.pyplot as plt

In [124]:
hep.style.use("CMS")
params = {'axes.labelsize': 20,
          'axes.titlesize': 20,
          'legend.fontsize':20, 
          'lines.markersize':5}
plt.rcParams.update(params)

In [3]:
dr_ylabel = r"$d\sigma \: [pb] \; / \; d\Delta R \: [bin]$"
avg_ylabel = r"$d\sigma \: [pb] \; / \; d p_T \: [GeV]$"
l0pt_ylabel = r"$d\sigma \: [pb] \; / \; d p_T \: [GeV]$"

In [138]:
fin1 = "LHCEFT_1410_S1_SM.pkl.gz"
fin2 = "LHCEFT_1410_S2_SM.pkl.gz"
fin3 = "LHCEFT_1410_noEFT_SM.pkl.gz"
hists1 = utils.get_hist_from_pkl(fin1, allow_empty=False)
hists2 = utils.get_hist_from_pkl(fin2, allow_empty=False)
hists3 = utils.get_hist_from_pkl(fin3, allow_empty=False)
print(hists1)

{'avg_top_pt': Hist(Regular(50, 0, 500, name='average top $p_T$ [GeV]'), storage=Weight()) # Sum: WeightedSum(value=11.632, variance=0.0744905) (WeightedSum(value=11.6475, variance=0.0746004) with flow), 'l0pt': Hist(Regular(50, 0, 500, name='leading lepton $p_T$ [GeV]'), storage=Weight()) # Sum: WeightedSum(value=11.6465, variance=0.0746004) (WeightedSum(value=11.6475, variance=0.0746004) with flow), 'dr_leps': Hist(Regular(30, 0, 6, name='$\\Delta R$ (leading lepton, subleading lepton)'), storage=Weight()) # Sum: WeightedSum(value=11.6475, variance=0.0746004), 'mtt': Hist(Regular(80, 0, 2000, name='invariant mass of tops'), storage=Weight()) # Sum: WeightedSum(value=11.6328, variance=0.0745132) (WeightedSum(value=11.6475, variance=0.0746004) with flow), 'njets': Hist(Regular(10, 0, 10, name='njets'), storage=Weight()) # Sum: WeightedSum(value=11.6475, variance=0.0746004)}


In [126]:
# h1 = hists1['avg_top_pt']
# h2 = hists2['avg_top_pt']
# h3 = hists3['avg_top_pt']
# fig, ax = plt.subplots(1,1)
# hep.histplot(h1, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (sample 1)")
# hep.histplot(h2, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (sample 2)")
# hep.histplot(h3, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (SM)")
# ax.legend(loc = 'upper right', fontsize = 'small')
# ax.set_ylabel(avg_ylabel, fontsize='medium')
# plt.figtext(0.78, 0.89,"(13 TeV)", fontsize = 'small')
# plt.figtext(0.14, 0.89, r"$pp \rightarrow t\bar{t} \rightarrow l^+ \nu_l b \;\; l^- \bar{\nu_l} \bar{b}$", fontsize='small')
# plt.figtext(0.25, 0.83, "Reweighted to SM", fontsize='small')
# figname = "LHCEFT_SM_avgTopPt.pdf"
# # fig.savefig(figname)
# print("plot saved to: ", figname)
# # plt.close(fig)

In [177]:
h1 = hists1['avg_top_pt']
h2 = hists2['avg_top_pt']
h3 = hists3['avg_top_pt']

fig, (ax, rax) = plt.subplots(
        nrows=2,
        ncols=1,
        figsize=(10,12),
        gridspec_kw={"height_ratios": (3, 1)},
        sharex=True
    )

fig.subplots_adjust(hspace=.1)

hep.histplot(h1, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (sample 1)")
hep.histplot(h2, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (sample 2)")
hep.histplot(h3, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (SM)")
ax.legend(loc = 'upper right', fontsize = 'small')
ax.set_ylabel(avg_ylabel, fontsize='medium')
ax.set_xlabel("")
plt.figtext(0.78, 0.89,"(13 TeV)", fontsize = 'small')
plt.figtext(0.14, 0.89, r"$pp \rightarrow t\bar{t} \rightarrow l^+ \nu_l b \;\; l^- \bar{\nu_l} \bar{b}$", fontsize='small')
plt.figtext(0.25, 0.83, "Reweighted to SM", fontsize='small')

# num1 = h1.values()[:-10]
# denom1 = h3.values()[:-10]
# centers1 = h1.axes.centers[0][:-10]
# rax.scatter(centers1, np.divide(num1, denom1), linewidths=0.5)

# num2 = h2.values()[:-10]
# centers2 = h2.axes.centers[0][:-10]
# rax.scatter(centers2, np.divide(num2, denom1))

num1 = h1.values()
denom1 = h3.values()
centers1 = h1.axes.centers[0]
rax.scatter(centers1, np.divide(num1, denom1), linewidths=0.5)

num2 = h2.values()
centers2 = h2.axes.centers[0]
rax.scatter(centers2, np.divide(num2, denom1))

rax.set_ylabel("Ratio EFT/SM", fontsize='small')
rax.set_xlabel(r"average top $p_T$ [GeV]", fontsize='small')
rax.set_ylim([0, 2])

# fig.show()
figname = "redoLHCEFT_SM_avgtoppt.pdf"
fig.savefig(figname)
print("plot saved to: ", figname)
plt.close(fig)

/tmp/ipykernel_3605748/2427249649.py:37: RuntimeWarning: divide by zero encountered in divide
  rax.scatter(centers1, np.divide(num1, denom1), linewidths=0.5)
/tmp/ipykernel_3605748/2427249649.py:41: RuntimeWarning: divide by zero encountered in divide
  rax.scatter(centers2, np.divide(num2, denom1))


plot saved to:  redoLHCEFT_SM_avgtoppt.pdf


In [178]:
h1 = hists1['l0pt']
h2 = hists2['l0pt']
h3 = hists3['l0pt']

fig, (ax, rax) = plt.subplots(
        nrows=2,
        ncols=1,
        figsize=(10,12),
        gridspec_kw={"height_ratios": (3, 1)},
        sharex=True
    )
fig.subplots_adjust(hspace=.1)

hep.histplot(h1, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (sample 1)")
hep.histplot(h2, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (sample 2)")
hep.histplot(h3, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (SM)")
ax.legend(loc = 'upper right', fontsize = 'small')
ax.set_ylabel(l0pt_ylabel, fontsize='medium')
ax.set_xlabel("")

plt.figtext(0.78, 0.89,"(13 TeV)", fontsize = 'small')
plt.figtext(0.14, 0.89, r"$pp \rightarrow t\bar{t} \rightarrow l^+ \nu_l b \;\; l^- \bar{\nu_l} \bar{b}$", fontsize='small')
plt.figtext(0.27, 0.83, "Reweighted to SM", fontsize='small')

num1 = h1.values()
denom1 = h3.values()
centers1 = h1.axes.centers[0]
rax.scatter(centers1, np.divide(num1, denom1), linewidths=0.5)

num2 = h2.values()
centers2 = h2.axes.centers[0]
rax.scatter(centers2, np.divide(num2, denom1))

rax.set_ylabel("Ratio EFT/SM", fontsize='small')
rax.set_xlabel(r"leading lepton $p_T$ [GeV]", fontsize='small')
rax.set_ylim([0, 3])

# fig.show()
figname = "redoLHCEFT_SM_l0pt.pdf"
fig.savefig(figname)
print("plot saved to: ", figname)
plt.close(fig)

plot saved to:  redoLHCEFT_SM_l0pt.pdf


/tmp/ipykernel_3605748/2115146082.py:28: RuntimeWarning: divide by zero encountered in divide
  rax.scatter(centers1, np.divide(num1, denom1), linewidths=0.5)
/tmp/ipykernel_3605748/2115146082.py:28: RuntimeWarning: invalid value encountered in divide
  rax.scatter(centers1, np.divide(num1, denom1), linewidths=0.5)
/tmp/ipykernel_3605748/2115146082.py:32: RuntimeWarning: divide by zero encountered in divide
  rax.scatter(centers2, np.divide(num2, denom1))
/tmp/ipykernel_3605748/2115146082.py:32: RuntimeWarning: invalid value encountered in divide
  rax.scatter(centers2, np.divide(num2, denom1))


In [179]:
h1 = hists1['dr_leps']
h2 = hists2['dr_leps']
h3 = hists3['dr_leps']

fig, (ax, rax) = plt.subplots(
        nrows=2,
        ncols=1,
        figsize=(10,12),
        gridspec_kw={"height_ratios": (3, 1)},
        sharex=True
    )
fig.subplots_adjust(hspace=.1)

hep.histplot(h1, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (sample 1)")
hep.histplot(h2, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (sample 2)")
hep.histplot(h3, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (SM)")

ax.legend(loc = 'upper right', fontsize = 18)
ax.set_ylabel(dr_ylabel, fontsize='medium')
ax.set_xlabel("")

plt.figtext(0.78, 0.89,"(13 TeV)", fontsize = 'small')
plt.figtext(0.14, 0.89, r"$pp \rightarrow t\bar{t} \rightarrow l^+ \nu_l b \;\; l^- \bar{\nu_l} \bar{b}$", fontsize='small')
plt.figtext(0.15, 0.83, "Reweighted to SM", fontsize='small')

num1 = h1.values()
denom1 = h3.values()
centers1 = h1.axes.centers[0]
rax.scatter(centers1, np.divide(num1, denom1), linewidths=0.5)

num2 = h2.values()
centers2 = h2.axes.centers[0]
rax.scatter(centers2, np.divide(num2, denom1))

rax.set_ylabel("Ratio EFT/SM", fontsize='small')
rax.set_xlabel(r"$\Delta$ R (leading lepton, subleading lepton)", fontsize='small')
# rax.set_ylim([0, 3])

# fig.show()
figname = "redoLHCEFT_SM_drleps.pdf"
fig.savefig(figname)
print("plot saved to: ", figname)
plt.close(fig)

plot saved to:  redoLHCEFT_SM_drleps.pdf


/tmp/ipykernel_3605748/3695836344.py:29: RuntimeWarning: divide by zero encountered in divide
  rax.scatter(centers1, np.divide(num1, denom1), linewidths=0.5)
/tmp/ipykernel_3605748/3695836344.py:29: RuntimeWarning: invalid value encountered in divide
  rax.scatter(centers1, np.divide(num1, denom1), linewidths=0.5)
/tmp/ipykernel_3605748/3695836344.py:33: RuntimeWarning: divide by zero encountered in divide
  rax.scatter(centers2, np.divide(num2, denom1))
/tmp/ipykernel_3605748/3695836344.py:33: RuntimeWarning: invalid value encountered in divide
  rax.scatter(centers2, np.divide(num2, denom1))


In [180]:
fin4 = "LHCEFT_1410_S1_pt1.pkl.gz"
fin5 = "LHCEFT_1410_S2_pt1.pkl.gz"
fin6 = "LHCEFT_1410_noEFT_SM.pkl.gz"
hists4 = utils.get_hist_from_pkl(fin4, allow_empty=False)
hists5 = utils.get_hist_from_pkl(fin5, allow_empty=False)
hists6 = utils.get_hist_from_pkl(fin6, allow_empty=False)

In [181]:
h1 = hists4['avg_top_pt']
h2 = hists5['avg_top_pt']
h3 = hists6['avg_top_pt']

fig, (ax, rax) = plt.subplots(
        nrows=2,
        ncols=1,
        figsize=(10,12),
        gridspec_kw={"height_ratios": (3, 1)},
        sharex=True
    )

fig.subplots_adjust(hspace=.1)

hep.histplot(h1, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (sample 1)")
hep.histplot(h2, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (sample 2)")
hep.histplot(h3, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (SM)")
ax.legend(loc = 'upper right', fontsize = 'small')
ax.set_ylabel(avg_ylabel, fontsize='medium')
ax.set_xlabel("")
plt.figtext(0.78, 0.89,"(13 TeV)", fontsize = 'small')
plt.figtext(0.14, 0.89, r"$pp \rightarrow t\bar{t} \rightarrow l^+ \nu_l b \;\; l^- \bar{\nu_l} \bar{b}$", fontsize='small')
plt.figtext(0.25, 0.83, "Reweighted to Pt2", fontsize='small')

num1 = h1.values()
denom1 = h3.values()
centers1 = h1.axes.centers[0]
rax.scatter(centers1, np.divide(num1, denom1))

num2 = h2.values()
centers2 = h2.axes.centers[0]
rax.scatter(centers2, np.divide(num2, denom1))

rax.set_ylabel("Ratio EFT/SM", fontsize='small')
rax.set_xlabel(r"average top $p_T$ [GeV]", fontsize='small')
rax.set_ylim([0, 12])

fig.show()

# fig.show()
figname = "redoLHCEFT_pt2_avgtoppt.pdf"
fig.savefig(figname)
print("plot saved to: ", figname)
plt.close(fig)

plot saved to:  redoLHCEFT_pt2_avgtoppt.pdf


/tmp/ipykernel_3605748/1149237736.py:28: RuntimeWarning: divide by zero encountered in divide
  rax.scatter(centers1, np.divide(num1, denom1))
/tmp/ipykernel_3605748/1149237736.py:32: RuntimeWarning: divide by zero encountered in divide
  rax.scatter(centers2, np.divide(num2, denom1))
/tmp/ipykernel_3605748/1149237736.py:38: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  fig.show()


In [182]:
h1 = hists4['l0pt']
h2 = hists5['l0pt']
h3 = hists6['l0pt']

fig, (ax, rax) = plt.subplots(
        nrows=2,
        ncols=1,
        figsize=(10,12),
        gridspec_kw={"height_ratios": (3, 1)},
        sharex=True
    )
fig.subplots_adjust(hspace=.1)

hep.histplot(h1, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (sample 1)")
hep.histplot(h2, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (sample 2)")
hep.histplot(h3, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (SM)")
ax.legend(loc = 'upper right', fontsize = 'small')
ax.set_ylabel(l0pt_ylabel, fontsize='medium')
ax.set_xlabel("")

plt.figtext(0.78, 0.89,"(13 TeV)", fontsize = 'small')
plt.figtext(0.14, 0.89, r"$pp \rightarrow t\bar{t} \rightarrow l^+ \nu_l b \;\; l^- \bar{\nu_l} \bar{b}$", fontsize='small')
plt.figtext(0.25, 0.83, "Reweighted to Pt2", fontsize='small')

num1 = h1.values()
denom1 = h3.values()
centers1 = h1.axes.centers[0]
rax.scatter(centers1, np.divide(num1, denom1), linewidths=0.5)

num2 = h2.values()
centers2 = h2.axes.centers[0]
rax.scatter(centers2, np.divide(num2, denom1))

rax.set_ylabel("Ratio EFT/SM", fontsize='small')
rax.set_xlabel(r"leading lepton $p_T$ [GeV]", fontsize='small')
rax.set_ylim([0, 20])

# fig.show()
figname = "redoLHCEFT_pt2_l0pt.pdf"
fig.savefig(figname)
print("plot saved to: ", figname)
plt.close(fig)

plot saved to:  redoLHCEFT_pt2_l0pt.pdf


/tmp/ipykernel_3605748/3984088244.py:28: RuntimeWarning: divide by zero encountered in divide
  rax.scatter(centers1, np.divide(num1, denom1), linewidths=0.5)
/tmp/ipykernel_3605748/3984088244.py:28: RuntimeWarning: invalid value encountered in divide
  rax.scatter(centers1, np.divide(num1, denom1), linewidths=0.5)
/tmp/ipykernel_3605748/3984088244.py:32: RuntimeWarning: divide by zero encountered in divide
  rax.scatter(centers2, np.divide(num2, denom1))
/tmp/ipykernel_3605748/3984088244.py:32: RuntimeWarning: invalid value encountered in divide
  rax.scatter(centers2, np.divide(num2, denom1))


In [183]:
h1 = hists4['dr_leps']
h2 = hists5['dr_leps']
h3 = hists6['dr_leps']

fig, (ax, rax) = plt.subplots(
        nrows=2,
        ncols=1,
        figsize=(10,12),
        gridspec_kw={"height_ratios": (3, 1)},
        sharex=True
    )
fig.subplots_adjust(hspace=.1)

hep.histplot(h1, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (sample 1)")
hep.histplot(h2, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (sample 2)")
hep.histplot(h3, ax=ax, stack=False, yerr=True, label=r"LO+1 jet (SM)")

ax.legend(loc = 'upper right', fontsize = 18)
ax.set_ylabel(dr_ylabel, fontsize='medium')
ax.set_xlabel("")

plt.figtext(0.78, 0.89,"(13 TeV)", fontsize = 'small')
plt.figtext(0.14, 0.89, r"$pp \rightarrow t\bar{t} \rightarrow l^+ \nu_l b \;\; l^- \bar{\nu_l} \bar{b}$", fontsize='small')
plt.figtext(0.25, 0.83, "Reweighted to Pt2", fontsize='small')

num1 = h1.values()
denom1 = h3.values()
centers1 = h1.axes.centers[0]
rax.scatter(centers1, np.divide(num1, denom1), linewidths=0.5)

num2 = h2.values()
centers2 = h2.axes.centers[0]
rax.scatter(centers2, np.divide(num2, denom1))

rax.set_ylabel("Ratio EFT/SM", fontsize='small')
rax.set_xlabel(r"$\Delta$ R (leading lepton, subleading lepton)", fontsize='small')
rax.set_ylim([0, 10])

# fig.show()
figname = "redoLHCEFT_pt_drleps.pdf"
fig.savefig(figname)
print("plot saved to: ", figname)
plt.close(fig)

plot saved to:  redoLHCEFT_pt_drleps.pdf


/tmp/ipykernel_3605748/3971123945.py:29: RuntimeWarning: divide by zero encountered in divide
  rax.scatter(centers1, np.divide(num1, denom1), linewidths=0.5)
/tmp/ipykernel_3605748/3971123945.py:29: RuntimeWarning: invalid value encountered in divide
  rax.scatter(centers1, np.divide(num1, denom1), linewidths=0.5)
/tmp/ipykernel_3605748/3971123945.py:33: RuntimeWarning: divide by zero encountered in divide
  rax.scatter(centers2, np.divide(num2, denom1))
/tmp/ipykernel_3605748/3971123945.py:33: RuntimeWarning: invalid value encountered in divide
  rax.scatter(centers2, np.divide(num2, denom1))
